In [2]:
import pyprind
import pandas as pd
from string import punctuation
import re
import numpy as np

df = pd.read_csv('movie_data.csv', encoding='utf-8')


In [3]:
from collections import Counter

counts = Counter()
pbar = pyprind.ProgBar(len(df['review']), title='Counting words occurrences')
for i, review in enumerate(df['review']):
    text = ''.join([c if c not in punctuation else ' ' + c + ' '
                    for c in review]).lower()
    df.loc[i, 'review'] = text
    pbar.update()
    counts.update(text.split())

Counting words occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:40


In [4]:
word_counts = sorted(counts, key=counts.get, reverse=True)
print(word_counts[:5])
word_to_int = {word: ii for ii, word in enumerate(word_counts, 1)}

mapped_reviews = []
pbar = pyprind.ProgBar(len(df['review']), title='Map reviews to ints')
for review in df['review']:
    mapped_reviews.append([word_to_int[word] for word in review.split()])
    pbar.update()

Map reviews to ints


['the', '.', ',', 'and', 'a']


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


In [5]:
sequence_length = 200
sequences = np.zeros((len(mapped_reviews), sequence_length), dtype=int)

for i, row in enumerate(mapped_reviews):
    review_arr = np.array(row)
    sequences[i, -len(row):] = review_arr[-sequence_length:]

In [6]:
X_train = sequences[:25000, :]
y_train = df.loc[:25000, 'sentiment'].values
X_test = sequences[25000:, :]
y_test = df.loc[25000:, 'sentiment'].values

In [7]:
np.random.seed(123)


def create_batch_generator(x, y=None, batch_size=64):
    n_batches = len(x) // batch_size
    x = x[:n_batches * batch_size]
    if y is not None:
        y = y[:n_batches * batch_size]
    for ii in range(0, len(x), batch_size):
        if y is not None:
            yield x[ii:ii + batch_size], y[ii:ii + batch_size]
        else:
            yield x[ii:ii + batch_size]


In [8]:
import tensorflow as tf


class SentimentRNN:

    def __init__(self, n_words, seq_len=200, lstm_size=256, num_layers=1,
                 batch_size=64, learning_rate=0.0001, embed_size=200):
        self.n_words = n_words
        self.seq_len = seq_len
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embed_size = embed_size

        self.g = tf.Graph()
        with self.g.as_default():
            tf.compat.v1.set_random_seed(123)
            self.build()
            self.saver = tf.compat.v1.train.Saver()
            self.init_op = tf.compat.v1.global_variables_initializer()

    def build(self):
        tf_x = tf.compat.v1.placeholder(tf.int32,
                                        shape=(self.batch_size, self.seq_len),
                                        name='tf_x')
        tf_y = tf.compat.v1.placeholder(tf.float32,
                                        shape=self.batch_size,
                                        name='tf_y')
        tf_keepprob = tf.compat.v1.placeholder(tf.float32,
                                               name='tf_keepprob')

        # 埋め込み層を作成
        embedding = tf.Variable(tf.compat.v1.random_uniform((self.n_words, self.embed_size),
                                                            minval=-1, maxval=1),
                                name='embedding')
        embed_x = tf.nn.embedding_lookup(embedding, tf_x,
                                         name='embeded_x')

        cells = tf.compat.v1.nn.rnn_cell.MultiRNNCell(
            [tf.compat.v1.nn.rnn_cell.DropoutWrapper(
                tf.compat.v1.nn.rnn_cell.BasicLSTMCell(self.lstm_size),
                output_keep_prob=tf_keepprob)
                for _ in range(self.num_layers)])

        self.initial_state = cells.zero_state(self.batch_size, tf.float32)
        print('  << initial state >> ', self.initial_state)

        lstm_outputs, self.final_state = tf.compat.v1.nn.dynamic_rnn(
            cells, embed_x, initial_state=self.initial_state)

        print('\n << lstm_output >> ', lstm_outputs)
        print('\n << final state >> ', self.final_state)

        logits = tf.compat.v1.layers.dense(inputs=lstm_outputs[:, -1],
                                           units=1, activation=None,
                                           name='logits')

        logits = tf.squeeze(logits, name='logits_squeezed')
        print('\n << logits      >> ', logits)

        y_proba = tf.nn.sigmoid(logits, name='probabilities')
        predictions = {
            'probabilities': y_proba,
            'labels': tf.cast(tf.round(y_proba), tf.int32, name='labels')
        }

        cost = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_y,
                                                    logits=logits),
            name='cost')

        optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.minimize(cost, name='train_op')

    def train(self, X_train, y_train, num_epochs):
        with tf.compat.v1.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            iteration = 1
            for epoch in range(num_epochs):
                state = sess.run(self.initial_state)

                for batch_x, batch_y in create_batch_generator(
                        X_train, y_train, self.batch_size):
                    feed = {'tf_x:0': batch_x,
                            'tf_y:0': batch_y,
                            'tf_keepprob:0': 0.5,
                            self.initial_state: state}
                    loss, _, state = sess.run(
                        ['cost:0', 'train_op', self.final_state],
                        feed_dict=feed)

                    if iteration % 20 == 0:
                        print('Epoch: %d/%d Iteration: %d | Train loss: %.5f' %
                              (epoch + 1, num_epochs, iteration, loss))
                    iteration += 1

                if (epoch + 1) % 10 == 0:
                    self.saver.save(sess, 'model/sentiment-%d.ckpt' % epoch)

    def predict(self, X_data, return_proba=False):
        preds = []
        with tf.compat.v1.Session(graph=self.g) as sess:
            self.saver.restore(sess, tf.compat.v1.train.latest_checkpoint('./model/'))
            test_state = sess.run(self.initial_state)
            for ii, batch_x in enumerate(create_batch_generator(
                    X_data, None, batch_size=self.batch_size), 1):
                feed = {'tf_x:0': batch_x, 'tf_keepprob:0': 1.0,
                        self.initial_state: test_state}

                if return_proba:
                    pred, test_state = sess.run(
                        ['probabilities:0', self.final_state],
                        feed_dict=feed)
                else:
                    pred, test_state = sess.run(
                        ['labels:0', self.final_state],
                        feed_dict=feed)

                preds.append(pred)

        return np.concatenate(preds)


In [10]:
n_words = max(list(word_to_int.values())) + 1

rnn = SentimentRNN(n_words=n_words,
                   seq_len=sequence_length,
                   embed_size=256,
                   lstm_size=128,
                   num_layers=1,
                   batch_size=100,
                   learning_rate=0.001)

  << initial state >>  (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(100, 128) dtype=float32>),)
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\users\1023d\.virtualenvs\pythonmachinelearning-qwvwlql2\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:708: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
c:\users\1023d\.virtualenvs\pythonmachinelearning-qwvwlql2\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '



 << lstm_output >>  Tensor("rnn/transpose_1:0", shape=(100, 200, 128), dtype=float32)

 << final state >>  (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Identity_4:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Identity_5:0' shape=(100, 128) dtype=float32>),)

 << logits      >>  Tensor("logits_squeezed:0", shape=(100,), dtype=float32)


c:\users\1023d\.virtualenvs\pythonmachinelearning-qwvwlql2\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
c:\users\1023d\.virtualenvs\pythonmachinelearning-qwvwlql2\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
rnn.train(X_train, y_train, num_epochs=20)

Epoch: 1/20 Iteration: 20 | Train loss: 0.69299
Epoch: 1/20 Iteration: 40 | Train loss: 0.61028
Epoch: 1/20 Iteration: 60 | Train loss: 0.65630
Epoch: 1/20 Iteration: 80 | Train loss: 0.49794
Epoch: 1/20 Iteration: 100 | Train loss: 0.49802
Epoch: 1/20 Iteration: 120 | Train loss: 0.50132
Epoch: 1/20 Iteration: 140 | Train loss: 0.53061
Epoch: 1/20 Iteration: 160 | Train loss: 0.43533
Epoch: 1/20 Iteration: 180 | Train loss: 0.52311
Epoch: 1/20 Iteration: 200 | Train loss: 0.42699
Epoch: 1/20 Iteration: 220 | Train loss: 0.39022
Epoch: 1/20 Iteration: 240 | Train loss: 0.45892
Epoch: 2/20 Iteration: 260 | Train loss: 0.33109
Epoch: 2/20 Iteration: 280 | Train loss: 0.39500
Epoch: 2/20 Iteration: 300 | Train loss: 0.43369
Epoch: 2/20 Iteration: 320 | Train loss: 0.42166
Epoch: 2/20 Iteration: 340 | Train loss: 0.40529
Epoch: 2/20 Iteration: 360 | Train loss: 0.31282
Epoch: 2/20 Iteration: 380 | Train loss: 0.28152
Epoch: 2/20 Iteration: 400 | Train loss: 0.39239
Epoch: 2/20 Iteration: 4

In [ ]:
preds = rnn.predict(X_test)
y_true = y_test[:len(preds)]
print('Test Acc.: %.3f' % (np.sum(preds == y_true) / len(y_true)))

In [ ]:
proba = rnn.predict(X_test, return_proba=True)

In [1]:
import numpy as np

with open('pg2265.txt', 'r', encoding='utf-8') as f:
    text = f.read()

text = text[15858:]
chars = set(text)
char2int = {ch: i for i, ch in enumerate(chars)}
int2char = dict(enumerate(chars))
text_ints = np.array([char2int[ch] for ch in text], dtype=np.int32)

In [2]:
def reshape_data(sequence, batch_size, num_steps):
    tot_batch_length = batch_size * num_steps
    num_batches = int(len(sequence) / tot_batch_length)

    if num_batches * tot_batch_length + 1 > len(sequence):
        num_batches = num_batches - 1

    x = sequence[0: num_batches * tot_batch_length]
    y = sequence[1: num_batches * tot_batch_length + 1]

    x_batch_splits = np.split(x, batch_size)
    y_batch_splits = np.split(y, batch_size)

    x = np.stack(x_batch_splits)
    y = np.stack(y_batch_splits)

    return x, y

In [3]:
def create_batch_generator(data_x, data_y, num_steps):
    batch_size, tot_batch_length = data_x.shape
    num_batchs = int(tot_batch_length / num_steps)
    for b in range(num_batchs):
        yield (data_x[:, b * num_steps:(b + 1) * num_steps],
               data_y[:, b * num_steps:(b + 1) * num_steps])

In [4]:
def get_top_char(probas, char_size, top_n=5):
    p = np.squeeze(probas)
    p[np.argsort(p)[:-top_n]] = 0.0
    p = p / np.sum(p)
    ch_id = np.random.choice(char_size, 1, p=p)[0]
    return ch_id

In [12]:
import tensorflow as tf
import os


class CharRNN:

    def __init__(self, num_classes, batch_size=64, num_steps=100,
                 lstm_size=128, num_layers=1, learning_rate=0.001,
                 keep_prob=0.5, grad_clip=5, sampling=False):
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.learning_rate = learning_rate
        self.keep_prob = keep_prob
        self.grad_clip = grad_clip

        self.g = tf.Graph()
        with self.g.as_default():
            tf.compat.v1.set_random_seed(123)

            self.build(sampling=sampling)
            self.saver = tf.compat.v1.train.Saver()
            self.init_op = tf.compat.v1.global_variables_initializer()

    def build(self, sampling):
        if sampling:
            batch_size, num_steps = 1, 1
        else:
            batch_size = self.batch_size
            num_steps = self.num_steps

        tf_x = tf.compat.v1.placeholder(tf.int32,
                                        shape=[batch_size, num_steps],
                                        name='tf_x')
        tf_y = tf.compat.v1.placeholder(tf.int32,
                                        shape=[batch_size, num_steps],
                                        name='tf_y')
        tf_keepprob = tf.compat.v1.placeholder(tf.float32,
                                               name='tf_keepprob')

        x_onehot = tf.one_hot(tf_x, depth=self.num_classes)
        y_onehot = tf.one_hot(tf_y, depth=self.num_classes)

        cells = tf.compat.v1.nn.rnn_cell.MultiRNNCell(
            [tf.compat.v1.nn.rnn_cell.DropoutWrapper(
                tf.compat.v1.nn.rnn_cell.BasicLSTMCell(self.lstm_size),
                output_keep_prob=tf_keepprob) for _ in range(self.num_layers)])

        self.initial_state = cells.zero_state(batch_size, tf.float32)
        lstm_outputs, self.final_state = tf.compat.v1.nn.dynamic_rnn(
            cells, x_onehot, initial_state=self.initial_state)

        print(' << lstm_outputs >> ', lstm_outputs)

        seq_output_reshaped = tf.reshape(lstm_outputs,
                                         shape=[-1, self.lstm_size],
                                         name='seq_output_reshaped')

        logits = tf.compat.v1.layers.dense(inputs=seq_output_reshaped,
                                           units=self.num_classes,
                                           activation=None,
                                           name='logits')

        proba = tf.nn.softmax(logits, name='probabilities')

        y_reshaped = tf.reshape(y_onehot,
                                shape=[-1, self.num_classes],
                                name='y_reshaped')
        cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped),
            name='cost')

        tvars = tf.compat.v1.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                          self.grad_clip)

        optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.apply_gradients(zip(grads, tvars),
                                             name='train_op')

    def train(self, train_x, train_y, num_epochs, ckpt_dir='./model/'):
        if not os.path.exists(ckpt_dir):
            os.mkdir(ckpt_dir)

        with tf.compat.v1.Session(graph=self.g) as sess:
            sess.run(self.init_op)

            n_batches = int(train_x.shape[1] / self.num_steps)
            iterations = n_batches * num_epochs
            for epoch in range(num_epochs):
                new_state = sess.run(self.initial_state)
                loss = 0

                bgen = create_batch_generator(train_x, train_y, self.num_steps)
                for b, (batch_x, batch_y) in enumerate(bgen, 1):
                    iteration = epoch * n_batches + b
                    feed = {'tf_x:0': batch_x, 'tf_y:0': batch_y,
                            'tf_keepprob:0': self.keep_prob,
                            self.initial_state: new_state}
                    batch_cost, _, new_state = sess.run(
                        ['cost:0', 'train_op', self.final_state],
                        feed_dict=feed)

                    if iteration % 10 == 0:
                        print('Epoch %d/%d Iteration %d| Training loss: %.4f' %
                              (epoch + 1, num_epochs, iteration, batch_cost))

                self.saver.save(sess,
                                os.path.join(ckpt_dir, 'language_modeling.ckpt'))

    def sample(self, output_length, ckpt_dir, starter_seq='The '):
        observed_seq = [ch for ch in starter_seq]
        with tf.compat.v1.Session(graph=self.g) as sess:
            self.saver.restore(sess,
                               tf.train.latest_checkpoint(ckpt_dir))

            new_state = sess.run(self.initial_state)
            for ch in starter_seq:
                x = np.zeros((1, 1))
                x[0, 0] = char2int[ch]
                feed = {'tf_x:0': x, 'tf_keepprob:0': 1.0,
                        self.initial_state: new_state}
                proba, new_state = sess.run(
                    ['probabilities:0', self.final_state],
                    feed_dict=feed)

                ch_id = get_top_char(proba, len(chars))
                observed_seq.append(int2char[ch_id])

            for i in range(output_length):
                x[0, 0] = ch_id
                feed = {'tf_x:0': x, 'tf_keepprob:0': 1.0,
                        self.initial_state: new_state}
                proba, new_state = sess.run(
                    ['probabilities:0', self.final_state],
                    feed_dict=feed)

                ch_id = get_top_char(proba, len(chars))
                observed_seq.append(int2char[ch_id])

        return ''.join(observed_seq)

    def summary(self):
        with self.g.as_default():
            file_writer = tf.compat.v1.summary.FileWriter(logdir='./logs', graph=self.g)
            file_writer.flush()


In [6]:
batch_size = 64
num_steps = 100
train_x, train_y = reshape_data(text_ints, batch_size, num_steps)

In [13]:
rnn = CharRNN(num_classes=len(chars), batch_size=batch_size)

c:\users\1023d\.virtualenvs\pythonmachinelearning-qwvwlql2\lib\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:708: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
c:\users\1023d\.virtualenvs\pythonmachinelearning-qwvwlql2\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
c:\users\1023d\.virtualenvs\pythonmachinelearning-qwvwlql2\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` inst

 << lstm_outputs >>  Tensor("rnn/transpose_1:0", shape=(64, 100, 128), dtype=float32)


In [14]:
rnn.summary()


In [ ]:
rnn.train(train_x, train_y, num_epochs=100, ckpt_dir='./model-100/')

In [11]:
del rnn
np.rondom.seed(123)
rnn = CharRNN(len(chars), sampling=True)
print(rnn.sample(ckpt_dir='./model-100/', output_length=500))

In [6]:
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


train_x shape: (64, 2500)
train_y shape: (64, 2500)


In [12]:
train_x[:10]

array([[63, 12, 26, ..., 12, 21, 41],
       [29, 26, 30, ..., 30, 64, 22],
       [26, 27, 30, ..., 49, 32, 12],
       ...,
       [30, 39, 21, ..., 23, 49, 32],
       [39, 30, 24, ..., 21, 41, 26],
       [17, 26, 15, ..., 39, 18, 18]])

In [13]:
text_ints[60:70]

array([21, 41, 53, 21, 41,  4, 22, 30, 21, 53])

160000